In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [8]:
# from zipfile import ZipFile
# from urllib.request import urlopen
# import os
# URL = 'https://www.kaggle.com/PromptCloudHQ/imdb-data/downloads/imdb-data-from-2006-to-2016.zip/1'

# url = urlopen(URL)
# output = open('zipFile.zip', 'wb')    # note the flag:  "wb"        
# output.write(url.read())
# output.close()
# df = pd.read_csv('zipFile.zip')

# df = pd.read_csv(URL, compression='zip', header=0, sep=',', quotechar='"')
# df = pd.read_csv('https://raw.githubusercontent.com/Thinkful-Ed/data-201-resources/master/ESS_practice_data/ESSdata_Thinkful.csv')

In [13]:
# Let's read in the data set

DataFilePath = '../../../../Datafiles'

df = pd.read_csv(DataFilePath + '/imdb-data-from-2006-to-2016.zip', 
                 compression='zip', header=0, sep=',', quotechar='"')

In [14]:
df.describe

<bound method NDFrame.describe of      Rank                                    Title  \
0       1                  Guardians of the Galaxy   
1       2                               Prometheus   
2       3                                    Split   
3       4                                     Sing   
4       5                            Suicide Squad   
5       6                           The Great Wall   
6       7                               La La Land   
7       8                                 Mindhorn   
8       9                       The Lost City of Z   
9      10                               Passengers   
10     11  Fantastic Beasts and Where to Find Them   
11     12                           Hidden Figures   
12     13                                Rogue One   
13     14                                    Moana   
14     15                                 Colossal   
15     16                  The Secret Life of Pets   
16     17                            Hacksaw Rid